In [ ]:
from kfp import dsl
# 定义
@dsl.component(base_image='python:3.9')
def square(x: float) -> float:
    print(f'square: {x}')
    return x ** 2

@dsl.component(base_image='python:3.9')
def add(x: float, y: float) -> float:
    print(f'add number: x={x}, y={y}')
    return x + y

@dsl.component(base_image='python:3.9')
def square_root(x: float) -> float:
    print(f'square_root: {x}')
    return x ** .5

@dsl.pipeline
def pythagorean(a: float, b: float) -> float:
    a_sq_task = square(x=a)
    a_sq_task.set_caching_options(False)

    b_sq_task = square(x=b)
    b_sq_task.set_caching_options(False)

    sum_task = add(x=a_sq_task.output, y=b_sq_task.output)
    sum_task.set_caching_options(False)

    root_task = square_root(x=sum_task.output)
    root_task.set_caching_options(False)

    return root_task.output

# 编译
from kfp import compiler

compiler.Compiler().compile(pythagorean, 'demo-pythagorean-pipeline.yaml')

# 上传/创建

from kfp.client import Client

client = Client(host='http://ml-pipeline.kubeflow:8888',
                namespace="kubeflow-user-example-com",
                existing_token="ACCESS_TOKEN",
                verify_ssl=False)

pipeline_info = client.upload_pipeline(
    pipeline_package_path='demo-pythagorean-pipeline.yaml',
    pipeline_name='demo-pythagorean-pipeline',
    description='This is a simple demo',
    namespace='kubeflow-user-example-com'
)
print(f"Pipeline uploaded! ID: {pipeline_info}")

# 运行
from kfp.client import Client

client = Client(host='http://ml-pipeline.kubeflow:8888',
                namespace="kubeflow-user-example-com",
                existing_token="ACCESS_TOKEN",
                verify_ssl=False)
run = client.create_run_from_pipeline_package(
    'demo-pythagorean-pipeline.yaml',
    arguments={
        'a': 3,
        'b': 4
    },
    enable_caching=False,
)
